In [ ]:
import pandas as pd
import numpy as np

In [4]:
num_rows = 1000
# create a dictionary of lists
data = {
    "x_1": np.random.normal(0, 1, num_rows),
    "x_2": np.random.normal(0, 1, num_rows),
    "x_3": np.random.normal(0, 1, num_rows),
    "x_4": np.random.normal(0, 1, num_rows),
    "x_5": np.random.normal(0, 1, num_rows),
}
beta = [0.5, 1, 1, 1, 1, 1]

# create a data frame from the dictionary
df = pd.DataFrame(data)
df["comb"] = beta[0] + beta[1] * df["x_1"] + beta[2] * df["x_2"] + beta[3] * df["x_3"] + beta[4] * df["x_4"] + beta[5] * df["x_5"]
df["prob"] = 1 / (1 + np.exp(-df["comb"]))
df["y"] = np.random.binomial(1, df["prob"], num_rows)
df.head(n=10)

,x_1,x_2,x_3,x_4,x_5,comb,prob,y
0,-0.242727,-0.126961,-0.950901,0.887027,1.108322,1.174760,0.764004,0
1,1.840215,-0.801730,0.944097,-0.424921,1.304017,3.361678,0.966485,1
2,-0.212608,0.101093,0.099550,-1.044257,-0.477740,-1.033962,0.262317,0
3,-1.048325,-0.550464,-1.154686,0.149300,-0.740912,-2.845087,0.054936,1
4,0.433712,1.641720,1.305467,-0.355261,-0.552996,2.972643,0.951323,1
5,1.721412,-1.375633,-0.727881,-0.294980,-1.287203,-1.464284,0.187813,0
6,-0.129920,0.546185,2.156676,-1.104268,-0.047734,1.920939,0.872243,1
7,-0.483083,0.231386,0.619679,1.012785,-0.285588,1.595180,0.831344,1
8,0.091344,-0.011296,1.491290,-1.122272,0.462484,1.411550,0.804010,1
9,0.008367,0.472841,1.079308,0.668965,-1.092462,1.637020,0.837129,1


In [ ]:
def generate_data(num_rows):
    data = {
        "x_1": np.random.normal(0, 1, num_rows),
        "x_2": np.random.normal(0, 1, num_rows),
        "x_3": np.random.normal(0, 1, num_rows),
        "x_4": np.random.normal(0, 1, num_rows),
        "x_5": np.random.normal(0, 1, num_rows),
    }
    beta = [0.5, 1, 1, 1, 1, 1]

    # create a data frame from the dictionary
    df = pd.DataFrame(data)
    df["comb"] = (
        beta[0]
        + beta[1] * df["x_1"]
        + beta[2] * df["x_2"]
        + beta[3] * df["x_3"]
        + beta[4] * df["x_4"]
        + beta[5] * df["x_5"]
    )
    df["prob"] = 1 / (1 + np.exp(-df["comb"]))
    df["y"] = np.random.binomial(1, df["prob"], num_rows)
    y = df["y"]
    X = df.drop(columns=["y", "prob", "comb"])
    return X, y
    